In [1]:
# Magic commands
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [18]:
import os
import nest_asyncio
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore

# from pinecone.exceptions import PineconeApiException
# import google.generativeai as genai

# from llama_index.llms.gemini import Gemini
# from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, PromptTemplate, get_response_synthesizer
# from llama_index.core.retrievers import VectorIndexRetriever


In [6]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]


In [7]:
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)
client = OpenAI(
    # Defaults to os.environ.get("OPENAI_API_KEY")
model="gpt-4o-mini", temperature=0)

In [8]:
embedding = OpenAIEmbedding(model="text-embedding-ada-002")
Settings.llm = client
Settings.embed_model = embedding
Settings.chunk_size_limit = 1536 

In [9]:
nest_asyncio.apply()

In [10]:
docs = SimpleDirectoryReader('../data/raw', recursive=True).load_data()

In [11]:
docs[0]

Document(id_='df136f77-be80-41e7-81c8-4195eb6c2d7b', embedding=None, metadata={'file_path': 'c:\\Users\\timytapilla\\Documents\\BYU-Idaho\\Fall_semester_2024\\Applied Programming CSE 310\\Projects\\FamilySearchAI\\notebooks\\..\\data\\raw\\attaching-a-source-to.md', 'file_name': 'attaching-a-source-to.md', 'file_size': 1423, 'creation_date': '2024-09-24', 'last_modified_date': '2024-09-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='---\r\ntitle: "\r\n        Using Source Linker to attach a  record to multiple ancestors\r\n    "\r\ndate: "\r\n                June 06, 2024\r\n            "\r\nurl: "https://www.familysearch.org/en/help/helpcenter/article/attaching-a-source-to-multiple-people-in-family-tree"\r\n---\r\n\r', mimetype='text/plain', start

In [14]:
df = pd.DataFrame([d.metadata for d in docs])
df.head(10)

,file_path,file_name,file_size,creation_date,last_modified_date
0,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,attaching-a-source-to.md,1423,2024-09-24,2024-09-24
1,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,attaching-a-source-to.md,1423,2024-09-24,2024-09-24
2,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,creating-new-people-in.md,1616,2024-09-24,2024-09-24
3,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,creating-new-people-in.md,1616,2024-09-24,2024-09-24
4,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,detaching-sources-that-should.md,1556,2024-09-24,2024-09-24
5,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,detaching-sources-that-should.md,1556,2024-09-24,2024-09-24
6,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,detaching-sources-that-should.md,1556,2024-09-24,2024-09-24
7,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,detaching-sources-that-should.md,1556,2024-09-24,2024-09-24
8,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,determining-if-a-source.md,1891,2024-09-24,2024-09-24
9,c:\Users\timytapilla\Documents\BYU-Idaho\Fall_...,determining-if-a-source.md,1891,2024-09-24,2024-09-24


In [ ]:
pinecone_index = pinecone_client.Index("chat bot")
vector_store = PineconeVectorStore(pinecone_index )